In [2]:
from DataHandler import DataHandler

dataHandler = DataHandler(19)

data = dataHandler.loadData()

data.head(5)

,buying,maint,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [24]:
def genFrequentOne(data, min_sup=0.1, min_conf=0.5):
    frequnet_itemsets = []
    features = data.columns[:-1]
    class_values = data['class'].unique()
    for feature in features:
        for class_value in class_values:
            feature_value = data[feature].unique()
            for value in feature_value:
                count = data[(data[feature] == value) & (data['class'] == class_value)].shape[0]
                support = count/data.shape[0]
                confidnce = count/data[(data[feature] == value)].shape[0]
                if support >= min_sup and confidnce >= min_conf:
                    condition = [{feature: value}]
                    frequnet_itemsets.append({'condition': condition, 'class': class_value,
                                             'support': support, 'confidence': confidnce})
    return frequnet_itemsets


def calculate(data, itemest):
    condition = itemest['condition']
    class_value = itemest['class']

    condition_keys = [list(cond.keys())[0] for cond in condition]
    condition_values = [list(cond.values())[0] for cond in condition]

    for i in range(len(condition_keys)):
        condition_condition = data[condition_keys[i]] == condition_values[i]

    condition_support_count = data[condition_condition].shape[0]
    condition_condition_with_class = condition_condition & (data['class'] == class_value)
    itemset_support_count = data[condition_condition_with_class].shape[0]

    if condition_support_count == 0:
        return 0, 0

    support = itemset_support_count/data.shape[0]
    confidence = itemset_support_count/condition_support_count
    return support, confidence


freqent_items = genFrequentOne(data)

# for item in freqent_items:
#     print(item)


k = 2
for item_a in freqent_items:
    if len(item_a['condition']) == k-1:
        for item_b in freqent_items:
            if item_a['class'] == item_b['class'] and len(item_a['condition']) == len(item_b['condition']):
                # 確認至少有k-2個條件相同
                count = 0
                condest_a = item_a['condition']
                condest_b = item_b['condition']
                while count < k-2:
                    for cond_a in condest_a:
                        for cond_b in condest_b:
                            if cond_a.keys() == cond_b.keys() and cond_a.values() == cond_b.values():
                                count += 1
                                condest_a.remove(cond_a)
                                condest_b.remove(cond_b)
                                break
                # 確認最後一個條件不同
                if condest_a[0].keys() != condest_b[0].keys():
                    # 合併條件
                    new_condition = item_a['condition'].copy()
                    new_condition.append({list(condest_b[0].keys())[0]: list(condest_b[0].values())[0]})
                    candidate = {'condition': new_condition, 'class': item_a['class']}
                    # 計算support
                    sup, conf = calculate(data, candidate)
                    if sup >= 0.01 and conf >= 0.5 and conf >= item_a['confidence'] and conf >= item_b['confidence']:
                        candidate['support'] = sup
                        candidate['confidence'] = conf
                        freqent_items.append(candidate)